## Загружаем данные

In [1]:
import pandas as pd

In [63]:
import numpy as np

In [2]:
df_movies = pd.read_csv("../lecture-1/movies.csv")
df_ratings = pd.read_csv("../lecture-1/ratings.csv")

In [4]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## Строим рекомендацию на основе KNN

In [70]:
from tqdm import tqdm

In [71]:
tqdm.pandas()

In [72]:
df = df_movies

In [73]:
df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [74]:
df['genres_splitted'] = df.genres.str.split('|')

In [75]:
df['genres_spaced'] = df.progress_apply(lambda r: ' '.join(r['genres_splitted']), axis=1)

100%|██████████| 9742/9742 [00:00<00:00, 59660.37it/s]


In [76]:
df.head()

,movieId,title,genres,genres_splitted,genres_spaced
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",Comedy Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]",Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],Comedy


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
tfidf = TfidfVectorizer()

In [ ]:
features = tfidf.fit_transform(df['genres_spaced'])

In [81]:
columns = [(k, tfidf.vocabulary_[k]) for k in tfidf.vocabulary_]

In [82]:
columns = sorted(columns, key=lambda c: c[1])

In [83]:
columns = [c[0] for c in columns]

In [84]:
features = features.todense()

In [85]:
df_features = pd.DataFrame(features, columns=columns)

In [86]:
df_result = pd.concat((df, df_features), axis=1)

In [87]:
df_result.columns

Index(['movieId', 'title', 'genres', 'genres_splitted', 'genres_spaced',
       'action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'fi', 'film', 'genres', 'horror',
       'imax', 'listed', 'musical', 'mystery', 'no', 'noir', 'romance', 'sci',
       'thriller', 'war', 'western'],
      dtype='object')

In [89]:
df_result

,movieId,title,genres,genres_splitted,genres_spaced,action,adventure,animation,children,comedy,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[Adventure, Animation, Children, Comedy, Fantasy]",Adventure Animation Children Comedy Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[Adventure, Children, Fantasy]",Adventure Children Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,"[Comedy, Romance]",Comedy Romance,0.000000,0.000000,0.000000,0.000000,0.570915,...,0.0,0.0,0.000000,0.0,0.0,0.821009,0.000000,0.000000,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[Comedy, Drama, Romance]",Comedy Drama Romance,0.000000,0.000000,0.000000,0.000000,0.505015,...,0.0,0.0,0.000000,0.0,0.0,0.726241,0.000000,0.000000,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,[Comedy],Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,"[Action, Crime, Thriller]",Action Crime Thriller,0.549328,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.542042,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,"[Comedy, Romance]",Comedy Romance,0.000000,0.000000,0.000000,0.000000,0.570915,...,0.0,0.0,0.000000,0.0,0.0,0.821009,0.000000,0.000000,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children,"[Adventure, Children]",Adventure Children,0.000000,0.636699,0.000000,0.771112,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,9,Sudden Death (1995),Action,[Action],Action,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,"[Action, Adventure, Thriller]",Action Adventure Thriller,0.553065,0.629522,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.545730,0.0,0.0


In [91]:
df_result = df_result[[
       'movieId', 'title', 'action', 'adventure', 'animation',
       'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi',
       'film', 'genres', 'horror', 'imax', 'listed', 'musical', 'mystery',
       'no', 'noir', 'romance', 'sci', 'thriller', 'war', 'western'   
]]

In [92]:
df_result.head()

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,1,Toy Story (1995),0.0,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),0.0,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),0.0,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),0.0,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [93]:
len(df_result)

9742

In [94]:
from sklearn.neighbors import NearestNeighbors

In [95]:
nn = NearestNeighbors(n_neighbors=50)

In [96]:
nn.fit(df_result[['action', 'adventure', 'animation',
       'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi',
       'film', 'horror', 'imax', 'listed', 'musical', 'mystery',
       'no', 'noir', 'romance', 'sci', 'thriller', 'war', 'western']])

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=50, p=2, radius=1.0)

In [121]:
df_result

,movieId,title,action,adventure,animation,children,comedy,crime,documentary,drama,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,1,Toy Story (1995),0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,2,Jumanji (1995),0.000000,0.512361,0.000000,0.620525,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,3,Grumpier Old Men (1995),0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.821009,0.000000,0.000000,0.0,0.0
3,4,Waiting to Exhale (1995),0.000000,0.000000,0.000000,0.000000,0.505015,0.000000,0.0,0.466405,...,0.0,0.0,0.000000,0.0,0.0,0.726241,0.000000,0.000000,0.0,0.0
4,5,Father of the Bride Part II (1995),0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,6,Heat (1995),0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.542042,0.0,0.0
6,7,Sabrina (1995),0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.821009,0.000000,0.000000,0.0,0.0
7,8,Tom and Huck (1995),0.000000,0.636699,0.000000,0.771112,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,9,Sudden Death (1995),1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,10,GoldenEye (1995),0.553065,0.629522,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.545730,0.0,0.0


In [109]:
df_joined = df_ratings.merge(df_result, on='movieId')

In [97]:
user_id = 320

In [110]:
def get_last_seven_films(user_id):
    user_films = df_joined[df_joined['userId'] == user_id]
    user_films = user_films.sort_values('timestamp', ascending=False)
    last_7 = user_films.head(7)
    
    film_names = last_7['title'].values
    film_vectors = last_7[['action', 'adventure', 'animation',
       'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'fi',
       'film', 'horror', 'imax', 'listed', 'musical', 'mystery',
       'no', 'noir', 'romance', 'sci', 'thriller', 'war', 'western']].values
    
    return film_names, film_vectors

In [141]:
def get_user_recomendation_by_knn(intereseted_films):
    _, films = nn.kneighbors(mean_vector.reshape(1, -1))
    return df_movies.iloc[films[0]]['movieId'].values
    

In [142]:
names, vectors = get_last_seven_films(user_id)

In [143]:
get_user_recomendation_by_knn(np.mean(vectors, axis=0))

array([  2094,  75416, 122906,   2105, 122892,  65982, 122900, 122918,
         2642, 167370,  62383, 122898,   1831,    442,   4987, 122916,
         6959, 169982,    260,  31553,  61210, 135569, 173291,  53464,
         3793, 186587,   8865,  33493,  34405,   4941,   6882, 122912,
         1210,  59315,   1196, 111362,   6294,   2628, 136800,   3702,
        63433, 141544,    208, 135567, 142509, 130842,   3704,   2275,
        60514,   3959])

## Делаем SVD

In [145]:
import surprise as s

In [146]:
df_for_surprise = df_ratings[['userId', 'movieId', 'rating']]

In [147]:
df_for_surprise.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [148]:
reader = s.reader.Reader(rating_scale=(0.5, 5))

In [149]:
dataset = s.dataset.Dataset.load_from_df(df_for_surprise, reader)

In [153]:
dataset, _ = s.model_selection.train_test_split(dataset, test_size=0.01)

In [154]:
algorithm = s.SVD()

In [155]:
algorithm.fit(dataset)

## Итоговая рекомендация

In [165]:
def recomend(user_id):
    names, vectors = get_last_seven_films(user_id)
    knn_recomendations = get_user_recomendation_by_knn(np.mean(vectors, axis=0))
    knn_recomendations = pd.DataFrame(knn_recomendations, columns=['movieId'])
    knn_recomendations['Score'] = knn_recomendations.apply(lambda r: algorithm.predict(user_id, r['movieId']).est, axis=1)
    knn_recomendations = knn_recomendations.sort_values('Score', ascending=False)
    knn_recomendations = knn_recomendations.merge(df_movies, on='movieId')[['movieId', 'title', 'Score']]
    
    print("Last 7 films was: ")
    print(names)
    
    print("Recomendations")
    print(knn_recomendations)

In [166]:
recomend(user_id)

Last 7 films was: 
['Avatar (2009)' 'Gladiator (2000)' 'Star Trek (2009)' 'Iron Man (2008)'
 'Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku no Tobira) (2001)'
 'Appleseed (Appurushîdo) (2004)' 'District 13 (Banlieue 13) (2004)']
Recomendations
    movieId                                              title     Score
0       260          Star Wars: Episode IV - A New Hope (1977)  4.057997
1    122918                   Guardians of the Galaxy 2 (2017)  4.013018
2      1196  Star Wars: Episode V - The Empire Strikes Back...  3.902829
3     59315                                    Iron Man (2008)  3.891793
4     34405                                    Serenity (2005)  3.885210
5      1210  Star Wars: Episode VI - Return of the Jedi (1983)  3.873675
6    122912             Avengers: Infinity War - Part I (2018)  3.823201
7    122916                              Thor: Ragnarok (2017)  3.819670
8    111362                  X-Men: Days of Future Past (2014)  3.815303
9    122906              